In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calc_mutual_inductance(coil_1, coil_2, d, po=0, fi=0):
    mu0 = 4 * np.pi * 10 ** (-7)
    mutual_inductance = np.ones((len(coil_1), len(coil_2)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    for ri in range(len(coil_1)):
        for rj in range(len(coil_2)):
            M = 0
            for n in range(N):
                for k in range(K):
                    xk_xn = po + coil_1[ri] * np.cos(df2 * k) * np.cos(fi) - coil_2[rj] * np.cos(df1 * n)
                    yk_yn = coil_1[ri] * np.sin(df2 * k) * np.cos(fi) - coil_2[rj] * np.sin(df1 * n)
                    zk_zn = d + coil_1[ri] * np.cos(df2 * k) * np.sin(fi)
                    r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                    M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
            M *= mu0 * coil_1[ri] * coil_2[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    return np.sum(mutual_inductance)

def calc_self_inductance(coil, thin=0.001):
    mu0 = 4 * np.pi * 10 ** (-7)
    L = np.sum(mu0 * coil * (np.log(8 * coil / thin) - 7 / 4 + (thin ** 2) / (8 * coil ** 2) * (np.log(8 * coil / thin) + 1 / 3)))
    mutual_inductance = np.ones((len(coil), len(coil)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    d = 0
    po = 0
    fi = 0
    for ri in range(len(coil)):
        for rj in range(len(coil)):
            M = 0
            if ri != rj:
                for n in range(N):
                    for k in range(K):
                        xk_xn = po + coil[ri] * np.cos(df2 * k) * np.cos(fi) - coil[rj] * np.cos(df1 * n)
                        yk_yn = coil[ri] * np.sin(df2 * k) * np.cos(fi) - coil[rj] * np.sin(df1 * n)
                        zk_zn = d + coil[ri] * np.cos(df2 * k) * np.sin(fi)
                        r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                        M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
                M *= mu0 * coil[ri] * coil[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    M = np.sum(mutual_inductance)
    L += M
    return L

def calc_coupling_coefficient(coil_1, coil_2, d, po=0, fi=0):
    M = calc_mutual_inductance(coil_1, coil_2, d, po=po, fi=fi)
    L1 = calc_self_inductance(coil_1)
    L2 = calc_self_inductance(coil_2)
    k = M / (L1 * L2) ** 0.5
    return k

def mutation(start, finish, x, r):
    x = np.random.uniform(low = start if x - r < start else x - r,
                          high = finish if x + r > finish else x + r)
    return x

# между витками одной катушки не должно быть пересечений
def test_coils(coil, thin):
    # между витками 2 и 3 катушки есть пересечение
    if coil[1] > coil[2]:
        if (coil[1] - coil[2] - 2 * thin) < 0:
            return False
    elif coil[2] > coil[1]:
        if (coil[2] - coil[1] - 2 * thin) < 0:
            return False
    return True

In [3]:
def next_ascent_hill_climbing(coil_1, thin, d):
    good_mutation = []
    bad_mutation = []
    all_mutation = []
    
    Q = 2
    band = 0.005
    thr_k = 1e-4
    
    start = 0.03 + 2 * thin
    finish = 0.09 - 2 * thin
    coil_2 = np.array([0.03, start, finish, 0.09])
    print("Начальное значение катушки:", coil_2)
    i = 0
    while True:
        i += 1
        fit_k = calc_coupling_coefficient(coil_1, coil_2, d)
        flag = False
        for q in range(Q):
            coil_2q = coil_2.copy()
            coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
            # проверка на пересечение витков внутри катушки
            while not test_coils(coil_2q, thin):
                print(f"Есть пересечения внутри катушки! {coil_2q}")
                coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
            fit_kq = calc_coupling_coefficient(coil_1, coil_2q, d)
            if fit_kq > fit_k:
                coil_2 = coil_2q.copy()
                fit_k = fit_kq
                flag = True
                good_mutation.append((fit_kq, coil_2q))
            else:
                bad_mutation.append((fit_kq, coil_2q))
            all_mutation.append((fit_kq, coil_2q))
        print(f"iter: {i} k = {fit_k}, катушка = {coil_2}")
        if len(good_mutation) >= 2:
            if (good_mutation[-1][0] - good_mutation[-2][0]) / thr_k < 1:
                break
        if i == 100:
            print("Алгоритм сделал 100 итераций! Останавливаю алгоритм")
            break
    return good_mutation[-1][0], good_mutation[-1][1], len(good_mutation), len(bad_mutation), len(all_mutation)

In [4]:
info_nahc = []

coil_1 = np.linspace(0.028, 0.07, 4)
thin = 0.001
d = 0.005
N = 10

In [5]:
for i in range(N):
    print(f"{i} итерация запуска алгоритма next ascent hill climbing:", end='\n\n')
    info_nahc.append(next_ascent_hill_climbing(coil_1, thin, d))

0 итерация запуска алгоритма next ascent hill climbing:

Начальное значение катушки: [0.03  0.032 0.088 0.09 ]
iter: 1 k = 0.48492504777704015, катушка = [0.03       0.03258011 0.08379011 0.09      ]
iter: 2 k = 0.48704791169762146, катушка = [0.03       0.03298151 0.08379011 0.09      ]
iter: 3 k = 0.537787649253027, катушка = [0.03       0.03625372 0.07945697 0.09      ]
iter: 4 k = 0.5616961389397002, катушка = [0.03       0.03625372 0.0765819  0.09      ]
iter: 5 k = 0.5819538318195646, катушка = [0.03       0.04045413 0.0765819  0.09      ]
iter: 6 k = 0.6040503610495318, катушка = [0.03       0.04474023 0.07523087 0.09      ]
iter: 7 k = 0.6040503610495318, катушка = [0.03       0.04474023 0.07523087 0.09      ]
iter: 8 k = 0.6040503610495318, катушка = [0.03       0.04474023 0.07523087 0.09      ]
iter: 9 k = 0.6040503610495318, катушка = [0.03       0.04474023 0.07523087 0.09      ]
iter: 10 k = 0.6040503610495318, катушка = [0.03       0.04474023 0.07523087 0.09      ]
iter: 1

iter: 23 k = 0.6545411377698054, катушка = [0.03       0.05541179 0.06712009 0.09      ]
iter: 24 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 25 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 26 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 27 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 28 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 29 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 30 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 31 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 32 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 33 k = 0.6547071048187073, катушка = [0.03       0.05471374 0.06712009 0.09      ]
iter: 34 k = 0.654707

iter: 3 k = 0.5083842332597772, катушка = [0.03       0.03717335 0.08364918 0.09      ]
iter: 4 k = 0.5105194800354756, катушка = [0.03       0.03760791 0.08364918 0.09      ]
iter: 5 k = 0.521328009058102, катушка = [0.03       0.03983563 0.08364918 0.09      ]
iter: 6 k = 0.5355928907612117, катушка = [0.03       0.04466873 0.08364918 0.09      ]
iter: 7 k = 0.5703160673946568, катушка = [0.03       0.04466873 0.07931569 0.09      ]
iter: 8 k = 0.5755426023079826, катушка = [0.03       0.0476032  0.07909562 0.09      ]
iter: 9 k = 0.5852968178506508, катушка = [0.03       0.05233827 0.07909562 0.09      ]
iter: 10 k = 0.5893425709562786, катушка = [0.03       0.05407897 0.07909562 0.09      ]
iter: 11 k = 0.5914372957495112, катушка = [0.03       0.05618115 0.07909562 0.09      ]
iter: 12 k = 0.5914372957495112, катушка = [0.03       0.05618115 0.07909562 0.09      ]
iter: 13 k = 0.5928727826374244, катушка = [0.03       0.05618115 0.07890071 0.09      ]
iter: 14 k = 0.59287278263742

iter: 27 k = 0.6541891292423065, катушка = [0.03       0.05611761 0.06764874 0.09      ]
iter: 28 k = 0.6541891292423065, катушка = [0.03       0.05611761 0.06764874 0.09      ]
iter: 29 k = 0.6541891292423065, катушка = [0.03       0.05611761 0.06764874 0.09      ]
iter: 30 k = 0.6543231215057213, катушка = [0.03       0.05611761 0.06853009 0.09      ]
iter: 31 k = 0.6543231215057213, катушка = [0.03       0.05611761 0.06853009 0.09      ]
iter: 32 k = 0.6550091945494719, катушка = [0.03       0.0552402  0.06853009 0.09      ]
iter: 33 k = 0.6550091945494719, катушка = [0.03       0.0552402  0.06853009 0.09      ]
iter: 34 k = 0.6550091945494719, катушка = [0.03       0.0552402  0.06853009 0.09      ]
iter: 35 k = 0.6550091945494719, катушка = [0.03       0.0552402  0.06853009 0.09      ]
iter: 36 k = 0.6550091945494719, катушка = [0.03       0.0552402  0.06853009 0.09      ]
iter: 37 k = 0.6550091945494719, катушка = [0.03       0.0552402  0.06853009 0.09      ]
iter: 38 k = 0.655009

In [6]:
info_nahc

[(0.651718900118296,
  array([0.03      , 0.05187949, 0.06783779, 0.09      ]),
  16,
  22,
  38),
 (0.6551253311337486,
  array([0.03      , 0.05479814, 0.06813878, 0.09      ]),
  18,
  44,
  62),
 (0.6535108177055768,
  array([0.03     , 0.0530429, 0.0677285, 0.09     ]),
  18,
  14,
  32),
 (0.6551076357934165,
  array([0.03      , 0.05471374, 0.06806709, 0.09      ]),
  23,
  61,
  84),
 (0.6550191518158459,
  array([0.03      , 0.05522464, 0.06851723, 0.09      ]),
  25,
  41,
  66),
 (0.654860848673381,
  array([0.03      , 0.05567635, 0.06805603, 0.09      ]),
  20,
  48,
  68),
 (0.6418411788701543,
  array([0.03      , 0.05617001, 0.072209  , 0.09      ]),
  18,
  24,
  42),
 (0.6482851388805385,
  array([0.03      , 0.04548989, 0.05843307, 0.09      ]),
  16,
  26,
  42),
 (0.6551449569730758,
  array([0.03      , 0.05501943, 0.06802019, 0.09      ]),
  18,
  28,
  46),
 (0.6550252125148578,
  array([0.03      , 0.05493202, 0.06853009, 0.09      ]),
  24,
  70,
  94)]

In [8]:
sorted(info_nahc, key=lambda x: x[0], reverse=True)

[(0.6551449569730758,
  array([0.03      , 0.05501943, 0.06802019, 0.09      ]),
  18,
  28,
  46),
 (0.6551253311337486,
  array([0.03      , 0.05479814, 0.06813878, 0.09      ]),
  18,
  44,
  62),
 (0.6551076357934165,
  array([0.03      , 0.05471374, 0.06806709, 0.09      ]),
  23,
  61,
  84),
 (0.6550252125148578,
  array([0.03      , 0.05493202, 0.06853009, 0.09      ]),
  24,
  70,
  94),
 (0.6550191518158459,
  array([0.03      , 0.05522464, 0.06851723, 0.09      ]),
  25,
  41,
  66),
 (0.654860848673381,
  array([0.03      , 0.05567635, 0.06805603, 0.09      ]),
  20,
  48,
  68),
 (0.6535108177055768,
  array([0.03     , 0.0530429, 0.0677285, 0.09     ]),
  18,
  14,
  32),
 (0.651718900118296,
  array([0.03      , 0.05187949, 0.06783779, 0.09      ]),
  16,
  22,
  38),
 (0.6482851388805385,
  array([0.03      , 0.04548989, 0.05843307, 0.09      ]),
  16,
  26,
  42),
 (0.6418411788701543,
  array([0.03      , 0.05617001, 0.072209  , 0.09      ]),
  18,
  24,
  42)]

In [9]:
print(f"Лучший коэффициент связи {info_nahc[0][0]} у катушки {info_nahc[0][1]}")
print(f"Среднее количество итераций {sum([i[4] for i in info_nahc]) / len(info_nahc)}")
print(f"Среднее количество полезных мутаций {sum([i[2] for i in info_nahc]) / len(info_nahc)}")
print(f"Среднее количество бесполезных мутаций {sum([i[3] for i in info_nahc]) / len(info_nahc)}")

Лучший коэффициент связи 0.651718900118296 у катушки [0.03       0.05187949 0.06783779 0.09      ]
Среднее количество итераций 57.4
Среднее количество полезных мутаций 19.6
Среднее количество бесполезных мутаций 37.8
